In [1]:
# https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

import nltk
import pandas as pd

In [3]:
data_path = "C:\\Users\\arthur\\Documents\\Data Science Club\\cxc-2022\\NLP\\IMDB Dataset.csv"

raw_ds = pd.read_csv(data_path)
raw_ds.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
raw_ds["sentiment"].value_counts()

# perfectly balanced as all things should be

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [5]:
sentiment_dict = {
    "positive": 1,
    "negative": 0
}

raw_ds["sentiment_num"] = raw_ds["sentiment"].map(sentiment_dict)
raw_ds.head()

,review,sentiment,sentiment_num
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [6]:
import re

def cleanup(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'[^\w\s]', '', sentence)
    return sentence

cleanup(raw_ds["review"].iloc[1])

'a wonderful little production br br the filming technique is very unassuming very oldtimebbc fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece br br the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too you can truly see the seamless editing guided by the references to williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece a masterful production about one of the great masters of comedy and his life br br the realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears it plays on our knowledge and our senses particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwells murals decorating every surface are terribly well done'

In [9]:
import numpy as np

glove_path = "C:\\Users\\arthur\\Documents\\Data Science Club\\cxc-2022\\NLP\\glove.6B.50d.txt"
glove_dict = dict()
glove_word2idx = dict()
glove_vect = []
GLOVE_DIM = 50
with open(glove_path,'r', encoding="utf8") as f:
    for idx, line in enumerate(f):
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], np.float32)
        glove_dict[word] = vector
        glove_word2idx[word] = idx
        glove_vect.append(vector)
        
glove_vect = np.stack(glove_vect)

In [10]:
pad_embedding = np.zeros((1,GLOVE_DIM))   #embedding for '<pad>' token.
unk_embedding = np.mean(glove_vect, axis=0, keepdims=True)    #embedding for '<unk>' token.
glove_vect = np.vstack((glove_vect, pad_embedding, unk_embedding))

In [11]:
glove_vect.shape

(400002, 50)

In [12]:
def vectorize_glove(word):
    if word in glove_dict.keys():
        return glove_word2idx[word]
    else:
        return len(glove_vect) - 1 #400000
    
print("hello: {} \n\nkjnkejnv: {}".format(vectorize_glove("hello"), vectorize_glove("kjnkejnv")))

hello: 13075 

kjnkejnv: 400001


In [13]:
MAX_SENT_LEN = 50

def vectorize_sentence(sentence):
    sent_vec = []
    words = list(reversed(nltk.word_tokenize(sentence)))
    for i in range(MAX_SENT_LEN):
        if i < len(words):
            sent_vec.append(vectorize_glove(words[i]))
        else:
            sent_vec.append(len(glove_vect) - 2)
            
    return np.stack(sent_vec)

vectorize_sentence(cleanup("I love maths"))

array([ 37668,    835,     41, 400000, 400000, 400000, 400000, 400000,
       400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000,
       400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000,
       400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000,
       400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000,
       400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000,
       400000, 400000])

In [14]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

class SentimentDataset(Dataset):
    def __init__(self, isTest=False):
        self.raw_ds = pd.read_csv(data_path)
        sentiment_dict = {
            "positive": 1,
            "negative": 0
        }

        self.raw_ds["sentiment_num"] = self.raw_ds["sentiment"].map(sentiment_dict)
        
        train_ds, test_ds = train_test_split(self.raw_ds, test_size=0.2, random_state=42)
        if isTest:
            self.ds = test_ds
        else:
            self.ds = train_ds

    def __getitem__(self,idx):
        x = self.ds["review"].iloc[idx]
        y = self.ds["sentiment_num"].iloc[idx]
        return vectorize_sentence(cleanup(x)), y
    
    def __len__(self):
        return len(self.ds)
    
train_sent = SentimentDataset()
test_sent = SentimentDataset(False)

train_iter = DataLoader(train_sent, batch_size=40, shuffle=True)
test_iter = DataLoader(test_sent, batch_size=40)

c:\Users\arthur\miniconda3\envs\nlp_workshop\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import torch
import torch.nn as nn

class BiLSTM(torch.nn.Module):  
    def __init__(self, 
                 in_dim, 
                 feature_dim = 50,
                 hidden_dim = 256,
                 out_dim = 2):
        super(BiLSTM,self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(400000 + 3, 50).from_pretrained(torch.tensor(glove_vect, dtype=torch.float64))
        self.lstm = torch.nn.LSTM(feature_dim, hidden_dim, num_layers=2, bidirectional=True, batch_first=True)
        self.dropout = torch.nn.Dropout(0.3) 
        self.dense = torch.nn.Linear(hidden_dim * 2, out_dim)
        
    def forward(self, x):   
        h0, c0 = self.init_hidden(x)
        x = self.dropout(self.embedding(x))
        x = torch.squeeze(torch.unsqueeze(x, 0))
        out, (hidden,_) = self.lstm(x, (h0,c0))
      
        x = self.dropout(out[:, -1, :])
        return self.dense(x)
    
    def init_hidden(self, x):
        h0 = torch.zeros(4, x.size(0), self.hidden_dim, dtype=torch.double).to(device)
        c0 = torch.zeros(4, x.size(0), self.hidden_dim, dtype=torch.double).to(device)
        return h0, c0

In [16]:
EPOCHS = 60
LR = 0.001

In [17]:
torch.cuda.manual_seed_all(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BiLSTM(MAX_SENT_LEN)
model = model.to(device).double()

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LR)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

In [19]:
train_acc = []
train_loss = []

test_acc = []
test_loss = []


for ep_idx in range(EPOCHS):
    model.train()
    corrects = 0
    total = 0
    losses = []
    for x, y in train_iter:
        x, y = x.to(device), y.to(device)
        og_out = model(x.type(torch.long))
        outputs = torch.squeeze(og_out[:, 0])
        loss = loss_fn(outputs, y.float())
        losses.append(loss.cpu().detach().numpy())
        preds = torch.nn.functional.softmax(og_out, dim=1)[:, 0].cpu().detach().numpy() > 0.5
        y_np = y.cpu().detach().numpy()

        corrects += np.sum(preds==y_np)
        total += len(x)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    train_acc.append(corrects/total)
    train_loss.append(np.mean(losses))
    
    scheduler.step()
    
    model.eval()
    corrects = 0
    total = 0
    losses = []
    with torch.no_grad():
        for x, y in test_iter:
            x, y = x.to(device), y.to(device)
            og_out = model(x.type(torch.long))
            outputs = torch.squeeze(og_out[:, 0])
            loss = loss_fn(outputs, y.float())
            losses.append(loss.cpu().detach().numpy())
            preds = torch.nn.functional.softmax(og_out, dim=1)[:, 0].cpu().detach().numpy() > 0.5
            y_np = y.cpu().detach().numpy()

            corrects += np.sum(preds==y_np)
            total += len(x)
            
    test_acc.append(corrects/total)
    test_loss.append(np.mean(losses))
    
    print("---EPOCH {}---".format(ep_idx))
    print("Train Acc {} || Loss {}".format(train_acc[-1], train_loss[-1]))
    print("Test Acc {} || Loss {}".format(test_acc[-1], test_loss[-1]))